<a href="https://colab.research.google.com/github/anddreza/poc_validacao_cenarios/blob/master/poc_validacao_cenarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install openai python-dotenv

In [40]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [41]:
from openai import OpenAI
client = OpenAI()
def ask_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = [
            { "role": "system", "content": "Você é um QA Automation Engineer Senior e deve responder tudo sempre em português"},
            { "role": "system", "content": "Você é uma LLM especializada em QA, BDD e Gherkin, responsável por padronizar cenários seguindo regras do time."},
            { "role": "system", "content": "Você é uma LLM especialista testes funcionais."},
            {"role": "system", "content": "Não alterar o significado da história"},
            { "role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

## Caso 1: Revisão dos cenários Gherkin

In [42]:
gherkin = """
  Feature: Login
  Scenario: Login realizado com sucesso
  Dado que o usuário está na página de login
  Quando o usuário informa credenciais válidas
  Então o usuário é redirecionado para o dashboard
  """

prompt = f"""
Revise o cenário Gherkin abaixo
- Sugira melhorias na escrita
- Gere cenários negativos
- Gere cenário de borda
- Aponte validações que estão faltando

{gherkin}

"""

print(ask_llm(prompt))

Claro! Vamos revisar o cenário Gherkin que você forneceu e sugerir melhorias, além de criar cenários negativos e de borda, e identificar validações que estão faltando.

### Cenário Gherkin Revisado

```gherkin
Feature: Login
  Scenario: Login realizado com sucesso
    Dado que o usuário está na página de login
    Quando o usuário informa credenciais válidas
    Então o usuário deve ser redirecionado para o dashboard
    E uma mensagem de boas-vindas deve ser exibida
```

### Sugestões de Melhorias na Escrita
1. **Clareza nas Expectativas**: Adicione uma validação de mensagem de boas-vindas para tornar o cenário mais completo.
2. **Consistência**: Use "deve" em vez de "é" para manter a consistência na forma como as expectativas são descritas.

### Cenários Negativos

```gherkin
Scenario: Login com credenciais inválidas
  Dado que o usuário está na página de login
  Quando o usuário informa credenciais inválidas
  Então o usuário deve ver uma mensagem de erro informando que as credencia

## Caso 2: Análise de falhas de teste

In [43]:
log = """
TimeoutExpection after 30 seconds
ElementNoInteractableExpection: elemento não clicável
"""

prompt = f"""
Analise o log de falha de teste abaixo
- Classifique a falha como: bug, instabilidade do teste, problema de ambiente ou erro de implementação
- Sugira próximos passos para investigação

Log:
{log}

"""

print(ask_llm(prompt))

Com base no log de falha de teste apresentado, podemos classificar a falha como:

**Classificação da falha:**  
- **Instabilidade do teste:** O erro "TimeoutException after 30 seconds" sugere que o teste não conseguiu interagir com um elemento dentro do tempo limite, o que pode ser um indicativo de instabilidade. Além disso, a "ElementNotInteractableException" indica que o elemento estava presente, mas não era interativo no momento da tentativa de clique.

**Próximos passos para investigação:**

1. **Verificar a visibilidade do elemento:** Confirme se o elemento que está sendo clicado está realmente visível e habilitado na interface no momento em que o teste tenta interagir com ele.

2. **Aumentar o tempo de espera:** Considere aumentar o tempo de espera para ver se o problema persiste. Isso pode ajudar a entender se a falha é causada por lentidão na aplicação.

3. **Revisar a lógica do teste:** Analise se a sequência de ações no teste está correta e se não há dependências que possam e

## Caso 3: Padronizar cenários verificando regras de negócio

In [44]:
gherkin = """
  Feature: Dispositivo
Scenario: Salvar dispositivo
When eu clico em salvar e vejo a mensagem de sucesso
Then o sistema funciona corretamente
  """

prompt = f"""
Sua tarefa é REESCREVER os cenários fornecidos seguindo rigorosamente as regras abaixo,
sem alterar o significado do teste.

REGRAS DE PADRONIZAÇÃO:

LINGUAGEM:
- Sempre usar terceira pessoa
- Usar o termo "o usuário"
- Nunca usar "eu", "nós", "a gente"

GIVEN:
- Define apenas contexto ou estado inicial
- Não deve conter ações do usuário

WHEN:
- Deve conter apenas UMA ação
- Não deve conter validações
- Não deve conter múltiplas ações conectadas por "e", "and"

THEN:
- Deve ser verificável e observável
- Deve validar comportamento, resposta ou estado do sistema
- Evitar termos vagos como "corretamente", "com sucesso", "normalmente"

FORMATAÇÃO:
- Manter estrutura Gherkin válida
- Não remover cenários
- Se um step violar as regras, reescreva-o

SAÍDA:
- Retorne apenas o cenário Gherkin reescrito
- Não explique as mudanças
- Não adicione comentários

{gherkin}

"""

print(ask_llm(prompt))

```gherkin
Feature: Dispositivo
Scenario: Salvar dispositivo
  Given o usuário está na tela de cadastro de dispositivo
  When o usuário clica em salvar
  Then o sistema exibe a mensagem de sucesso
```


## Caso 4: Caso de teste gerado a partir da história de usuário

In [45]:
gherkin = """
  Feature: Dispositivo
Scenario: Salvar dispositivo
When eu clico em salvar e vejo a mensagem de sucesso
Then o sistema funciona corretamente
  """

prompt = f"""
REGRAS GERAIS:
- Gerar cenários claros, objetivos e verificáveis
- Usar linguagem padronizada em terceira pessoa
- Usar sempre o termo "o usuário"
- Não usar "eu", "nós" ou "a gente"

ESTRUTURA DOS CENÁRIOS:
- Criar cenários de:
  - Happy path
  - Erro/negativo
  - Borda (valores mínimos, máximos ou limites)
- Cada cenário deve conter:
  - Given (contexto)
  - When (uma única ação)
  - Then (resultado verificável)

REGRAS DE QUALIDADE:
- Um cenário deve validar apenas um comportamento principal
- Then deve validar algo observável (mensagem, status, estado do sistema)
- Evitar termos vagos como "corretamente" ou "com sucesso"

FORMATO DE SAÍDA:
- Retornar apenas Gherkin válido
- Não explicar decisões
- Não adicionar comentários

HISTÓRIA DE USUÁRIO:
<<<
{gherkin}
>>>


"""

print(ask_llm(prompt))

```gherkin
Feature: Dispositivo

  Scenario: Salvar dispositivo - Happy path
    Given o usuário está na tela de cadastro de dispositivo
    When o usuário clica em salvar
    Then o sistema exibe a mensagem "Dispositivo salvo com sucesso"

  Scenario: Salvar dispositivo - Erro/negativo
    Given o usuário está na tela de cadastro de dispositivo
    When o usuário clica em salvar sem preencher os campos obrigatórios
    Then o sistema exibe a mensagem "Preencha todos os campos obrigatórios"

  Scenario: Salvar dispositivo - Borda
    Given o usuário está na tela de cadastro de dispositivo
    When o usuário insere um nome de dispositivo com 255 caracteres
    Then o sistema exibe a mensagem "Dispositivo salvo com sucesso"
```
